In [1]:
import pathlib

import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import chi2_contingency
from scipy.stats import false_discovery_control
from scipy.stats import mannwhitneyu
from shapely import Point
from behavioral_analysis.behavior_extraction.funcs import behavior_series_to_df, behavior_to_bout_df, bout_to_behavior_df, bout_df_to_transition_series
from behavioral_analysis.math_tools.angle_funcs import cart2pol
from behavioral_analysis.moseq_tools.syllable_funcs import load_group_dict_from_csv
from behavioral_analysis.pandas_tools.files import build_file_df
from behavioral_analysis.pandas_tools.filter import filter_series_by_modified_zscore
from behavioral_analysis.pandas_tools.pattern_extraction import get_pattern_match_df, match_df_to_chunks_df
from behavioral_analysis.pandas_tools.rle import rle_series
from behavioral_analysis.tracking_tools.funcs import extract_subtrack_df, normalize_track_df
from behavioral_analysis.utility.builtin_classes.objects import load_object, save_object
from statsmodels.formula.api import ols
from tqdm.cli import tqdm

d:\BITrusted\conda_envs\manuscript2024\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root_dir = pathlib.Path(r"I:\2024_manuscript")
video_dir = root_dir / "videos"
data_output_dir = root_dir / "data_output"
tracking_dir = root_dir / "tracking"
moseq_project_dir = root_dir / "keypoint_moseq"

In [3]:
# global parameters
px_per_mm = 0.8631346578366446  # see above
contact_threshold = 100  # mm

minimum_syllable_onset_frequency = 0.005

fps = 25.23
start_frame_limit = int(fps * 30)  # 30 seconds
end_frame_limit = int(fps * 60 * 20)  # 20 minutes

bl6_light_cycle_date_dict = {
    "norm": pd.to_datetime("19-03-24"),
    "reverse": pd.to_datetime("20-03-24"),
}

In [4]:
tracking_df = pd.concat({track_file.name.rsplit(".", 1)[0]: pd.read_csv(track_file, index_col=0, header=[0, 1]) for track_file in tracking_dir.glob("*.csv")}, axis=0, names=["track"])

In [5]:
moseq_df = load_object(data_output_dir / "moseq_df.pkl")

In [6]:
syll_frequencies = moseq_df[moseq_df["onset"]]["syllable"].value_counts(normalize=True)
relevant_syllables = syll_frequencies[syll_frequencies.ge(minimum_syllable_onset_frequency)].sort_index().index
relevant_syllables

Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
      dtype='int64', name='syllable')

In [7]:
onset_moseq_df = moseq_df.loc[moseq_df["onset"]]
onset_frame_proportions = onset_moseq_df.groupby(["context", "track"])["syllable"].value_counts(normalize=True).sort_index()

syll_pvalues = {}
for syll in relevant_syllables:
    syll_series = onset_frame_proportions.xs(syll, level="syllable")
    res = mannwhitneyu(syll_series.loc["Solitary"].values, syll_series.loc["Dyadic"].values)
    syll_pvalues[syll] = res.pvalue

syll_pvalue_series = pd.Series(syll_pvalues)
corr_syll_pvalue_series = pd.Series(false_discovery_control(syll_pvalue_series.values), index=syll_pvalue_series.index)
sig_syllables = corr_syll_pvalue_series[corr_syll_pvalue_series < 0.05].index
sig_syllables

Index([5, 7, 10, 12, 13, 18, 23, 30], dtype='int64')

# Supplementary Figure 1

## Panel A

In [8]:
# everything already generated in Step 3

## Panel B

In [9]:
# everything already generated in Step 3

## Panel C

In [10]:
sampled_syllable_trajectory_df = load_object(data_output_dir / "sampled_syllable_trajectory_df.pkl")

In [11]:
out_dir = pathlib.Path(r"C:\Users\rittmart\OneDrive - Boehringer Ingelheim\jupyter_notebooks\2024_Manuscript\plots_svg\trajectories")

syllable_example_stat_df_dict = {}
for syll in tqdm(sampled_syllable_trajectory_df.index.get_level_values("syllable").unique()):
    keypoint_df = sampled_syllable_trajectory_df.loc[syll].sort_index()
    keypoint_df = keypoint_df.drop(["tail_tip", "tail_center"], axis=1, level=0).drop("score", axis=1, level=1)

    median_df = keypoint_df.stack("keypoint_feature", future_stack=True).median(axis=1).unstack("keypoint_feature")
    delta_median = median_df.loc[15] - median_df.loc[-5]
    median_distance, median_direction = cart2pol(delta_median["x"], delta_median["y"])
    median_direction = np.abs(median_direction)

    viewing_direction_series = (keypoint_df["nose"] - keypoint_df["spine_cervical"]).apply(lambda r: cart2pol(r["x"], r["y"])[1], axis=1)
    viewing_direction_delta = np.abs(viewing_direction_series.loc[15] - viewing_direction_series.loc[-5])

    syllable_example_stat_df_dict[syll] = pd.Series(dict(median_distance=median_distance, median_direction=median_direction, viewing_direction_delta=viewing_direction_delta, sig_syll=syll in sig_syllables))
syllable_example_stat_df = pd.concat(syllable_example_stat_df_dict, axis=1, names=["syll"]).T
syllable_example_stat_df = syllable_example_stat_df.astype({"sig_syll": bool, })
syllable_example_stat_df

100%|██████████| 32/32 [00:00<00:00, 328.54it/s]


,median_distance,median_direction,viewing_direction_delta,sig_syll
syll,,,,
0,118.775409,0.460187,0.328843,False
1,10.57963,2.484382,0.989406,False
2,115.23701,0.85101,0.785908,False
3,3.970795,0.780752,0.76351,False
4,11.826725,2.989279,0.446708,False
5,176.165502,0.057234,0.130132,True
6,1.088831,0.854643,0.068779,False
7,10.492056,3.036815,0.048847,True
8,196.146703,0.115167,0.142832,False


In [12]:
for value in ["median_distance", "median_direction", "viewing_direction_delta"]:
    group_series = syllable_example_stat_df["sig_syll"].astype(bool)
    data_series = syllable_example_stat_df[value].astype(float)

    group1_series = data_series.loc[group_series]
    group1_series = filter_series_by_modified_zscore(group1_series).dropna()

    group2_series = data_series.loc[~group_series]
    group2_series = filter_series_by_modified_zscore(group2_series).dropna()

    print(value, mannwhitneyu(group1_series, group2_series))

median_distance MannwhitneyuResult(statistic=np.float64(50.0), pvalue=np.float64(0.14406916200019648))
median_direction MannwhitneyuResult(statistic=np.float64(39.0), pvalue=np.float64(0.03327077569569227))
viewing_direction_delta MannwhitneyuResult(statistic=np.float64(33.0), pvalue=np.float64(0.0047619862525313025))


In [13]:
save_object(syllable_example_stat_df, data_output_dir / "syllable_example_stat_df.pkl", overwrite=True)

# Supplementary Figure 2

## Panel A & B

In [14]:
behavior_df = behavior_series_to_df(moseq_df["syllable"])
bout_df = behavior_to_bout_df(behavior_df)
bout_df

bout_feature                                 end_frame  bout_length  \
track                           frame_index                           
1 - 21-02-24.analysis           756                757            2   
                                758                767           10   
                                768                770            3   
                                771                774            4   
                                775                790           16   
...                                                ...          ...   
7&10 - 20-03-24.analysis_track1 30208            30218           11   
                                30219            30245           27   
                                30246            30258           13   
                                30259            30268           10   
                                30269            30276            8   

bout_feature                                 behavior_name  
track                           frame_index                 
1 - 21-02-24.analysis           756                      1  
                                758                     13  
                                768                      9  
                                771                      0  
                                775                     17  
...                                                    ...  
7&10 - 20-03-24.analysis_track1 30208                   11  
                                30219                   18  
                                30246                   23  
                                30259                    0  
                                30269                    5  

[82988 rows x 3 columns]

In [15]:
save_object(behavior_df, data_output_dir / "behavior_df.pkl", overwrite=True)

In [16]:
removed_syll_bout_df = bout_df.copy()
removed_syll_bout_df["behavior_name"] = (~removed_syll_bout_df["behavior_name"].isin(relevant_syllables)).astype(int)
removed_syll_bout_df = behavior_to_bout_df(bout_to_behavior_df(removed_syll_bout_df))
removed_syll_bout_df = removed_syll_bout_df[removed_syll_bout_df["behavior_name"].eq(1)]
removed_syll_bout_df["dyadic_bout"] = removed_syll_bout_df.index.get_level_values("track").str.contains("track")
removed_syll_bout_df["randomized"] = False
removed_syll_bout_df

100%|██████████| 2/2 [00:00<00:00, 18.87it/s]


bout_feature                                 end_frame  bout_length  \
track                           frame_index                           
1 - 21-02-24.analysis           1179              1184            6   
                                3016              3031           16   
                                4140              4200           61   
                                4555              4780          226   
                                7962              7964            3   
...                                                ...          ...   
7&10 - 20-03-24.analysis_track1 27336            27483          148   
                                28809            29339          531   
                                29453            29473           21   
                                29583            29627           45   
                                30013            30048           36   

bout_feature                                 behavior_name  dyadic_bout  \
track                           frame_index                               
1 - 21-02-24.analysis           1179                     1        False   
                                3016                     1        False   
                                4140                     1        False   
                                4555                     1        False   
                                7962                     1        False   
...                                                    ...          ...   
7&10 - 20-03-24.analysis_track1 27336                    1         True   
                                28809                    1         True   
                                29453                    1         True   
                                29583                    1         True   
                                30013                    1         True   

bout_feature                                 randomized  
track                           frame_index              
1 - 21-02-24.analysis           1179              False  
                                3016              False  
                                4140              False  
                                4555              False  
                                7962              False  
...                                                 ...  
7&10 - 20-03-24.analysis_track1 27336             False  
                                28809             False  
                                29453             False  
                                29583             False  
                                30013             False  

[1212 rows x 5 columns]

In [17]:
rng = np.random.default_rng(232323)

def shuffle_bout_df(single_input_bout_df, random_state=None):
    first_frame_index = single_input_bout_df.sort_index().index[0]
    randomized_bout_df = single_input_bout_df.sample(frac=1.0, random_state=random_state).copy()
    randomized_bout_df["end_frame"] = randomized_bout_df["bout_length"].cumsum() + first_frame_index - 1
    randomized_bout_df = randomized_bout_df.set_index((randomized_bout_df["end_frame"] - randomized_bout_df["bout_length"] + 1).rename("frame_index"))
    return randomized_bout_df

randomized_bout_df = bout_df.groupby("track").apply(lambda x: shuffle_bout_df(x.droplevel("track"), random_state=rng))
randomized_bout_df

bout_feature                                 end_frame  bout_length  \
track                           frame_index                           
1 - 21-02-24.analysis           756                763            8   
                                764                768            5   
                                769                786           18   
                                787                787            1   
                                788                809           22   
...                                                ...          ...   
7&10 - 20-03-24.analysis_track1 30206            30212            7   
                                30213            30231           19   
                                30232            30240            9   
                                30241            30251           11   
                                30252            30276           25   

bout_feature                                 behavior_name  
track                           frame_index                 
1 - 21-02-24.analysis           756                      9  
                                764                      9  
                                769                     15  
                                787                      0  
                                788                     12  
...                                                    ...  
7&10 - 20-03-24.analysis_track1 30206                   15  
                                30213                   11  
                                30232                   16  
                                30241                    4  
                                30252                   13  

[82988 rows x 3 columns]

In [18]:
randomized_removed_syll_bout_df = randomized_bout_df.copy()
randomized_removed_syll_bout_df["behavior_name"] = (~randomized_removed_syll_bout_df["behavior_name"].isin(relevant_syllables)).astype(int)
randomized_removed_syll_bout_df = behavior_to_bout_df(bout_to_behavior_df(randomized_removed_syll_bout_df))
randomized_removed_syll_bout_df = randomized_removed_syll_bout_df[randomized_removed_syll_bout_df["behavior_name"].eq(1)]
randomized_removed_syll_bout_df["dyadic_bout"] = randomized_removed_syll_bout_df.index.get_level_values("track").str.contains("track")
randomized_removed_syll_bout_df["randomized"] = True
randomized_removed_syll_bout_df

100%|██████████| 2/2 [00:00<00:00, 17.57it/s]


bout_feature                                 end_frame  bout_length  \
track                           frame_index                           
1 - 21-02-24.analysis           884                898           15   
                                1069              1103           35   
                                1135              1196           62   
                                1212              1234           23   
                                1285              1318           34   
...                                                ...          ...   
7&10 - 20-03-24.analysis_track1 28961            28990           30   
                                29047            29058           12   
                                29144            29355          212   
                                29412            29415            4   
                                29925            29971           47   

bout_feature                                 behavior_name  dyadic_bout  \
track                           frame_index                               
1 - 21-02-24.analysis           884                      1        False   
                                1069                     1        False   
                                1135                     1        False   
                                1212                     1        False   
                                1285                     1        False   
...                                                    ...          ...   
7&10 - 20-03-24.analysis_track1 28961                    1         True   
                                29047                    1         True   
                                29144                    1         True   
                                29412                    1         True   
                                29925                    1         True   

bout_feature                                 randomized  
track                           frame_index              
1 - 21-02-24.analysis           884                True  
                                1069               True  
                                1135               True  
                                1212               True  
                                1285               True  
...                                                 ...  
7&10 - 20-03-24.analysis_track1 28961              True  
                                29047              True  
                                29144              True  
                                29412              True  
                                29925              True  

[3323 rows x 5 columns]

In [19]:
joint_removed_syll_bout_df = pd.concat([removed_syll_bout_df, randomized_removed_syll_bout_df], axis=0)
joint_removed_syll_bout_df

bout_feature                                 end_frame  bout_length  \
track                           frame_index                           
1 - 21-02-24.analysis           1179              1184            6   
                                3016              3031           16   
                                4140              4200           61   
                                4555              4780          226   
                                7962              7964            3   
...                                                ...          ...   
7&10 - 20-03-24.analysis_track1 28961            28990           30   
                                29047            29058           12   
                                29144            29355          212   
                                29412            29415            4   
                                29925            29971           47   

bout_feature                                 behavior_name  dyadic_bout  \
track                           frame_index                               
1 - 21-02-24.analysis           1179                     1        False   
                                3016                     1        False   
                                4140                     1        False   
                                4555                     1        False   
                                7962                     1        False   
...                                                    ...          ...   
7&10 - 20-03-24.analysis_track1 28961                    1         True   
                                29047                    1         True   
                                29144                    1         True   
                                29412                    1         True   
                                29925                    1         True   

bout_feature                                 randomized  
track                           frame_index              
1 - 21-02-24.analysis           1179              False  
                                3016              False  
                                4140              False  
                                4555              False  
                                7962              False  
...                                                 ...  
7&10 - 20-03-24.analysis_track1 28961              True  
                                29047              True  
                                29144              True  
                                29412              True  
                                29925              True  

[4535 rows x 5 columns]

In [20]:
save_object(joint_removed_syll_bout_df, data_output_dir / "joint_removed_syll_bout_df.pkl", overwrite=True)

## Panel C-F

In [21]:
joint_removed_syll_bout_df = load_object(data_output_dir / "joint_removed_syll_bout_df.pkl")

In [22]:
filtered_removed_syll_bout_df = joint_removed_syll_bout_df[joint_removed_syll_bout_df["bout_length"].ge(25)]

video_file_df = build_file_df(video_dir, r".*\\(?P<video_name>.*)$")
video_name_path_dict = video_file_df.set_index("video_name")["file_path"].to_dict()

known_tracks = filtered_removed_syll_bout_df.index.get_level_values("track").unique().to_series()
video_path_series = filtered_removed_syll_bout_df.index.get_level_values("track").unique().to_series()
video_path_series = video_path_series.str.rsplit(".", n=1, expand=True)[0] + ".mp4"
video_path_series = video_path_series.apply(video_name_path_dict.get).rename("video_path")
video_path_series.index = known_tracks

In [23]:
rng = np.random.default_rng(232323)
sampled_bouts = filtered_removed_syll_bout_df.groupby(["dyadic_bout", "randomized"]).sample(n=9, random_state=rng)
sampled_bouts["random_frame"] = sampled_bouts.apply(lambda x: np.random.randint(low=x.name[1], high=x["end_frame"]+1), axis=1)
sampled_bouts["video_path"] = video_path_series.loc[sampled_bouts.index.get_level_values("track")].values
sampled_bouts

,bout_feature,end_frame,bout_length,behavior_name,dyadic_bout,randomized,random_frame,video_path
track,frame_index,,,,,,,
25 - 21-02-24.analysis,21491,21526,36,1,False,False,21504,I:\2024_manuscript\videos\solitary\20240221_no...
10 - 21-02-24.analysis,8769,8908,140,1,False,False,8828,I:\2024_manuscript\videos\solitary\20240221_no...
1 - 21-02-24.analysis,26878,26944,67,1,False,False,26879,I:\2024_manuscript\videos\solitary\20240221_no...
7 - 21-02-24.analysis,22493,22574,82,1,False,False,22541,I:\2024_manuscript\videos\solitary\20240221_no...
1 - 21-02-24.analysis,11695,11742,48,1,False,False,11705,I:\2024_manuscript\videos\solitary\20240221_no...
16 - 23-02-24.analysis,27896,27926,31,1,False,False,27906,I:\2024_manuscript\videos\solitary\20240223_re...
7 - 21-02-24.analysis,25643,25893,251,1,False,False,25860,I:\2024_manuscript\videos\solitary\20240221_no...
1 - 21-02-24.analysis,28225,28444,220,1,False,False,28363,I:\2024_manuscript\videos\solitary\20240221_no...
22 - 23-02-24.analysis,13456,13534,79,1,False,False,13518,I:\2024_manuscript\videos\solitary\20240223_re...


In [24]:
bout_frame_df = sampled_bouts[["random_frame", "video_path", "dyadic_bout", "randomized"]]
bout_frame_df = bout_frame_df.reset_index("frame_index").set_index("random_frame", append=True)
bout_track_df = tracking_df.loc[bout_frame_df.index]
bout_track_df

keypoint_name                                  instance  ear_left              \
keypoint_feature                                  score     score           x   
track                            frame_index                                    
25 - 21-02-24.analysis           21504         9.467870  0.792204  216.403778   
10 - 21-02-24.analysis           8828          8.831480  0.754240  440.481079   
1 - 21-02-24.analysis            26879        10.141631  0.891972  156.441940   
7 - 21-02-24.analysis            22541         9.922803  0.901059   98.320000   
1 - 21-02-24.analysis            11705         9.988866  0.730532  415.717010   
16 - 23-02-24.analysis           27906         9.902504  0.790574  473.371704   
7 - 21-02-24.analysis            25860         5.765170  0.685124  406.211182   
1 - 21-02-24.analysis            28363         8.227659  0.880728  197.142532   
22 - 23-02-24.analysis           13518         9.035808  0.800161   62.137115   
25 - 21-02-24.analysis           14490         8.158693  0.749724  429.891205   
19 - 21-02-24.analysis           5834         10.247408  0.859416  165.915024   
1 - 23-02-24.analysis            17638        10.423676  0.878067   76.881577   
13 - 21-02-24.analysis           4565         10.175784  0.837588  476.091522   
                                 25304         9.513871  0.629096  268.561523   
19 - 23-02-24.analysis           25743        10.310858  0.829927  314.997253   
4 - 23-02-24.analysis            25350         9.545545  0.729031  316.465179   
                                 8894          8.973255  0.703859   62.679672   
25 - 23-02-24.analysis           20044         9.977942  0.823297  187.410782   
19&22 - 19-03-24.analysis_track1 17176        10.212926  0.780660  500.342224   
19&22 - 20-03-24.analysis_track0 9955          8.997421  0.739065  292.558777   
13&16 - 19-03-24.analysis_track0 26013        10.022003  0.585793  493.905273   
1&4 - 19-03-24.analysis_track0   1289          6.947768  0.906738  516.577576   
                                 24441         7.575420  0.863270  507.702637   
25&28 - 19-03-24.analysis_track0 5014          6.732671  0.747545  155.999481   
7&10 - 20-03-24.analysis_track0  17713        10.116173  0.873187  205.104416   
13&16 - 19-03-24.analysis_track0 22699         9.677300  0.951657  182.891815   
25&28 - 19-03-24.analysis_track0 8467          8.478456  0.727357  467.485992   
13&16 - 20-03-24.analysis_track0 21106        10.498815  0.905920  180.178116   
1&4 - 20-03-24.analysis_track0   25780         9.493644  0.945814  187.523010   
13&16 - 20-03-24.analysis_track0 24252        10.521082  0.725001  544.418335   
19&22 - 19-03-24.analysis_track0 3079          9.444794  0.814300  139.995132   
7&10 - 20-03-24.analysis_track0  7003          9.384424  0.745144  503.725769   
25&28 - 20-03-24.analysis_track1 2832          9.256953  0.626161  162.306305   
25&28 - 19-03-24.analysis_track0 26476        10.359437  0.863338  157.671509   
19&22 - 19-03-24.analysis_track1 27979        10.166805  0.836154  310.901917   
25&28 - 19-03-24.analysis_track1 28887         9.793737  0.761981  155.425217   

keypoint_name                                            ear_right  \
keypoint_feature                                       y     score   
track                            frame_index                         
25 - 21-02-24.analysis           21504        110.622246  0.775582   
10 - 21-02-24.analysis           8828         384.412964  0.763363   
1 - 21-02-24.analysis            26879        135.663589  0.858039   
7 - 21-02-24.analysis            22541        443.110413  0.825707   
1 - 21-02-24.analysis            11705         72.588310  0.820497   
16 - 23-02-24.analysis           27906         84.657143  0.817814   
7 - 21-02-24.analysis            25860        423.432953  0.652331   
1 - 21-02-24.analysis            28363        160.362198  0.721701   
22 - 23-02-24.analysis           13518        357.845123  0.875263   


In [25]:
save_object(bout_frame_df, data_output_dir / "bout_frame_df.pkl", overwrite=True)
save_object(bout_track_df, data_output_dir / "bout_track_df.pkl", overwrite=True)

# Two-way ANOVA

In [26]:
scored_contact_series = load_object(data_output_dir / "scored_active_contact_series.pkl")

In [27]:
renamed_contact_quartile_series = load_object(data_output_dir / "renamed_contact_quartile_series.pkl")
scored_contact_series = load_object(data_output_dir / "scored_active_contact_series.pkl")

In [28]:
scored_contact_moseq_df = moseq_df.join(scored_contact_series)
scored_contact_moseq_df["comparison_group"] = scored_contact_moseq_df["context"] + scored_contact_moseq_df["contact"].apply({True: ", Yes", False: ", No", np.nan: ""}.get)
plot_syllable_proportions = scored_contact_moseq_df[scored_contact_moseq_df["onset"]]
plot_syllable_proportions = plot_syllable_proportions.groupby(["comparison_group", "track"])["syllable"].value_counts(normalize=True).reset_index()
plot_syllable_proportions = plot_syllable_proportions[plot_syllable_proportions["comparison_group"].ne("Solitary")]
plot_syllable_proportions["contact"] = plot_syllable_proportions["comparison_group"].str.split(", ", expand=True)[1]
filtered_syllable_proportions = plot_syllable_proportions[plot_syllable_proportions["syllable"].isin(sig_syllables)]

_contact_quartile_moseq_df = moseq_df.join(renamed_contact_quartile_series)
_contact_quartile_moseq_df["comparison_group"] = _contact_quartile_moseq_df["context"] + _contact_quartile_moseq_df["contact_quartile"].apply(lambda x: ", " + x if not pd.isna(x) else "")
quart_plot_syllable_proportions = _contact_quartile_moseq_df[_contact_quartile_moseq_df["onset"]]
quart_plot_syllable_proportions = quart_plot_syllable_proportions.groupby(["comparison_group", "track"])["syllable"].value_counts(normalize=True).reset_index()
quart_plot_syllable_proportions = quart_plot_syllable_proportions[quart_plot_syllable_proportions["comparison_group"].ne("Solitary")]
quart_plot_syllable_proportions["quartile"] = quart_plot_syllable_proportions["comparison_group"].str.split(", ", expand=True)[1]
filtered_quart_syllable_proportions = quart_plot_syllable_proportions[quart_plot_syllable_proportions["syllable"].isin(sig_syllables)]

In [29]:
lm = ols(formula="proportion ~ C(syllable) * C(contact)", data=filtered_syllable_proportions).fit()
table = sm.stats.anova_lm(lm, typ=2) # Type 2 ANOVA DataFrame
table

,sum_sq,df,F,PR(>F)
C(syllable),0.033854,7.0,49.604544,1.656677e-46
C(contact),0.000497,1.0,5.095900,2.470379e-02
C(syllable):C(contact),0.001520,7.0,2.227487,3.204397e-02
Residual,0.029152,299.0,NaN,NaN


In [30]:
filtered_quart_syllable_proportions["syllable"].unique()

array([ 7, 13,  5, 10, 12, 18, 30, 23])

In [31]:
lm = ols(formula="proportion ~ C(syllable) * C(quartile)", data=filtered_quart_syllable_proportions).fit()
table = sm.stats.anova_lm(lm, typ=2) # Type 2 ANOVA DataFrame
table

,sum_sq,df,F,PR(>F)
C(syllable),0.070764,7.0,92.452276,3.518449e-91
C(quartile),0.001269,3.0,3.868191,9.292022e-03
C(syllable):C(quartile),0.019978,21.0,8.700506,8.874025e-24
Residual,0.065825,602.0,NaN,NaN


# Supplementary Figure 3

## Panel A

In [32]:
# data already generated in Step 3

## Panel B

In [33]:
# stats

behavior_series = moseq_df["syllable"]
behavior_df = behavior_series_to_df(behavior_series)
bout_df = behavior_to_bout_df(behavior_df)
transition_series = bout_df_to_transition_series(bout_df)
transition_series

100%|██████████| 40/40 [00:03<00:00, 10.69it/s]


track                            previous_behavior  next_behavior
1 - 21-02-24.analysis            1                  1                 0
                                                    13               21
                                                    9                32
                                                    0                 1
                                                    17                0
                                                                     ..
7&10 - 20-03-24.analysis_track1  62                 43                0
                                                    47                0
                                                    71                0
                                                    65                0
                                                    62                0
Name: transition_count, Length: 118540, dtype: int64

In [34]:
common_transitions = transition_series.groupby(["previous_behavior", "next_behavior"]).min().ge(1)
common_transitions = common_transitions[common_transitions]

full_relevant_index = pd.MultiIndex.from_product([relevant_syllables, relevant_syllables], names=["previous_behavior", "next_behavior"])
common_transitions = common_transitions.loc[full_relevant_index.intersection(common_transitions.index)].sort_index()
common_transitions

previous_behavior  next_behavior
0                  5                True
                   8                True
                   17               True
                   19               True
                   25               True
                                    ... 
26                 10               True
27                 22               True
28                 0                True
29                 8                True
31                 0                True
Name: transition_count, Length: 99, dtype: bool

In [35]:
transition_series_copy = transition_series[transition_series.ne(0)].rename("transition_count")
transition_series_copy = transition_series_copy.groupby("track").apply(lambda x: x.droplevel("track").reindex(full_relevant_index)).dropna()
transition_series_copy = transition_series_copy.groupby("track").size().rename("observed_transitions")
transition_series_copy = transition_series_copy.to_frame()
transition_series_copy["Context"] = transition_series_copy.index.str.contains("track")
transition_series_copy["Context"] = transition_series_copy["Context"].apply(lambda x: "Dyadic" if x else "Solitary")
transition_series_copy

,observed_transitions,Context
track,,
1 - 21-02-24.analysis,229,Solitary
1 - 23-02-24.analysis,252,Solitary
1&4 - 19-03-24.analysis_track0,250,Dyadic
1&4 - 19-03-24.analysis_track1,244,Dyadic
1&4 - 20-03-24.analysis_track0,249,Dyadic
1&4 - 20-03-24.analysis_track1,241,Dyadic
10 - 21-02-24.analysis,243,Solitary
10 - 23-02-24.analysis,219,Solitary
13 - 21-02-24.analysis,222,Solitary


In [36]:
g1 = transition_series_copy.loc[transition_series_copy["Context"].eq("Dyadic"), "observed_transitions"]
g2 = transition_series_copy.loc[transition_series_copy["Context"].eq("Solitary"), "observed_transitions"]

mannwhitneyu(g1, g2, alternative="greater")

MannwhitneyuResult(statistic=np.float64(278.0), pvalue=np.float64(0.017876162309093724))

In [37]:
save_object(transition_series_copy, data_output_dir / "syllable_transition_count_df.pkl", overwrite=True)

## Panel C

In [38]:
behavior_df = behavior_series_to_df(moseq_df["syllable"])
bout_df = behavior_to_bout_df(behavior_df)
bout_df

bout_feature                                 end_frame  bout_length  \
track                           frame_index                           
1 - 21-02-24.analysis           756                757            2   
                                758                767           10   
                                768                770            3   
                                771                774            4   
                                775                790           16   
...                                                ...          ...   
7&10 - 20-03-24.analysis_track1 30208            30218           11   
                                30219            30245           27   
                                30246            30258           13   
                                30259            30268           10   
                                30269            30276            8   

bout_feature                                 behavior_name  
track                           frame_index                 
1 - 21-02-24.analysis           756                      1  
                                758                     13  
                                768                      9  
                                771                      0  
                                775                     17  
...                                                    ...  
7&10 - 20-03-24.analysis_track1 30208                   11  
                                30219                   18  
                                30246                   23  
                                30259                    0  
                                30269                    5  

[82988 rows x 3 columns]

In [39]:
transition_series = bout_df_to_transition_series(bout_df)
transition_df = transition_series.unstack("next_behavior")
transition_df = transition_df.loc[pd.IndexSlice[:, relevant_syllables.values], relevant_syllables.values]
transition_df

100%|██████████| 40/40 [00:03<00:00, 10.27it/s]


next_behavior                                       0     1    2    3     4   \
track                           previous_behavior                              
1 - 21-02-24.analysis           0                  0.0   0.0  0.0  0.0   0.0   
                                1                  1.0   0.0  0.0  2.0   5.0   
                                2                  0.0   0.0  0.0  0.0   0.0   
                                3                  0.0   0.0  0.0  0.0  20.0   
                                4                  0.0  17.0  0.0  4.0   0.0   
...                                                ...   ...  ...  ...   ...   
7&10 - 20-03-24.analysis_track1 27                 0.0   0.0  0.0  3.0   0.0   
                                28                 8.0   0.0  2.0  0.0   3.0   
                                29                 1.0   0.0  0.0  0.0   0.0   
                                30                 0.0   0.0  3.0  0.0   0.0   
                                31                 9.0   0.0  0.0  0.0   0.0   

next_behavior                                        5     6    7     8   \
track                           previous_behavior                          
1 - 21-02-24.analysis           0                  80.0   0.0  0.0   1.0   
                                1                   0.0  10.0  0.0   0.0   
                                2                   0.0   0.0  0.0  55.0   
                                3                   0.0   0.0  7.0   0.0   
                                4                   0.0  43.0  0.0   0.0   
...                                                 ...   ...  ...   ...   
7&10 - 20-03-24.analysis_track1 27                  0.0   3.0  0.0   0.0   
                                28                  0.0   0.0  0.0   2.0   
                                29                  0.0   0.0  0.0   8.0   
                                30                  0.0   0.0  0.0   0.0   
                                31                  0.0   0.0  0.0   0.0   

next_behavior                                        9   ...    22   23   24  \
track                           previous_behavior        ...                   
1 - 21-02-24.analysis           0                   0.0  ...   0.0  1.0  0.0   
                                1                  32.0  ...   0.0  0.0  0.0   
                                2                   0.0  ...  25.0  0.0  2.0   
                                3                   0.0  ...   0.0  6.0  0.0   
                                4                   2.0  ...   0.0  0.0  0.0   
...                                                 ...  ...   ...  ...  ...   
7&10 - 20-03-24.analysis_track1 27                  0.0  ...   5.0  0.0  0.0   
                                28                  0.0  ...   0.0  4.0  7.0   
                                29                  0.0  ...   1.0  0.0  0.0   
                                30                  0.0  ...   0.0  5.0  0.0   
                                31                  0.0  ...   0.0  0.0  0.0   

next_behavior                                        25   26   27   28   29  \
track                           previous_behavior                             
1 - 21-02-24.analysis           0                  13.0  0.0  0.0  0.0  0.0   
                                1                   1.0  9.0  0.0  0.0  0.0   
                                2                   0.0  0.0  0.0  3.0  4.0   
                                3                   0.0  0.0  3.0  0.0  0.0   
                                4                   0.0  0.0  2.0  0.0  0.0   
...                                                 ...  ...  ...  ...  ...   
7&10 - 20-03-24.analysis_track1 27                  0.0  0.0  0.0  0.0  0.0   
                                28                  0.0  0.0  0.0  0.0  0.0   
                                29                  0.0  0.0  0.0  0.0  0.0   
                                30                  0.0  0.0

In [40]:
normed_transition_df = (transition_df.T / transition_df.T.sum(axis=0)).T

normed_transition_df = normed_transition_df.unstack("previous_behavior")
dyadic_tracks = normed_transition_df.index.get_level_values("track").str.contains("track")
normed_transition_df = normed_transition_df.groupby(dyadic_tracks).agg("median")
normed_transition_df.index.name = "dyadic_context"
normed_transition_df = normed_transition_df.stack("previous_behavior", future_stack=True)

normed_transition_df = (normed_transition_df.T / normed_transition_df.T.sum(axis=0)).T
normed_transition_df.sum(axis=1)

dyadic_context  previous_behavior
False           0                    1.0
                1                    1.0
                2                    1.0
                3                    1.0
                4                    1.0
                                    ... 
True            27                   1.0
                28                   1.0
                29                   1.0
                30                   1.0
                31                   1.0
Length: 64, dtype: float64

In [41]:
normed_transition_series = normed_transition_df.stack("next_behavior", future_stack=True).unstack("dyadic_context")
normed_transition_series = normed_transition_series[normed_transition_series.ne(0).all(axis=1)]
normed_transition_series = normed_transition_series.rename({False: "Solitary", True: "Dyadic"}, axis=1).rename_axis("Context", axis=1)
normed_transition_series

Context                          Solitary    Dyadic
previous_behavior next_behavior                    
0                 2              0.005805  0.008883
                  5              0.617426  0.571531
                  8              0.016690  0.019730
                  17             0.146397  0.131832
                  19             0.035124  0.042621
...                                   ...       ...
30                16             0.023093  0.030256
                  23             0.629819  0.656990
31                0              0.719482  0.725083
                  19             0.184355  0.130295
                  23             0.058296  0.077666

[234 rows x 2 columns]

In [42]:
save_object(normed_transition_series, data_output_dir / "normed_transition_series.pkl", overwrite=True)

## Panel D

In [43]:
sylls_needed_to_90_percent = normed_transition_df.apply(lambda x: x.sort_values(ascending=False).cumsum().ge(0.9).argmax()+1, axis=1)
sylls_needed_to_90_percent = sylls_needed_to_90_percent.rename("n90").to_frame()
sylls_needed_to_90_percent["sig_syll"] = sylls_needed_to_90_percent.index.get_level_values("previous_behavior").isin(sig_syllables)
sylls_needed_to_90_percent = sylls_needed_to_90_percent.rename({False: "Solitary", True: "Dyadic"}, axis=0, level=0).rename_axis(["Context", "previous_behavior"], axis=0)
sylls_needed_to_90_percent

n90  sig_syll
Context  previous_behavior               
Solitary 0                    3     False
         1                    6     False
         2                    3     False
         3                    8     False
         4                    5     False
...                         ...       ...
Dyadic   27                   5     False
         28                   7     False
         29                   6     False
         30                   3      True
         31                   3     False

[64 rows x 2 columns]

In [44]:
save_object(sylls_needed_to_90_percent, data_output_dir / "sylls_needed_to_90_percent.pkl", overwrite=True)

In [45]:
# Extract the four groups based on "Context" and "sig_syll"
solitary_sig = sylls_needed_to_90_percent[(sylls_needed_to_90_percent.index.get_level_values("Context") == "Solitary") & (sylls_needed_to_90_percent["sig_syll"])]
solitary_non_sig = sylls_needed_to_90_percent[(sylls_needed_to_90_percent.index.get_level_values("Context") == "Solitary") & (~sylls_needed_to_90_percent["sig_syll"])]
dyadic_sig = sylls_needed_to_90_percent[(sylls_needed_to_90_percent.index.get_level_values("Context") == "Dyadic") & (sylls_needed_to_90_percent["sig_syll"])]
dyadic_non_sig = sylls_needed_to_90_percent[(sylls_needed_to_90_percent.index.get_level_values("Context") == "Dyadic") & (~sylls_needed_to_90_percent["sig_syll"])]

# Perform Mann-Whitney U tests
results = {
    "Solitary Sig vs Non-Sig": mannwhitneyu(solitary_sig["n90"], solitary_non_sig["n90"]),
    "Dyadic Sig vs Non-Sig": mannwhitneyu(dyadic_sig["n90"], dyadic_non_sig["n90"]),
    "Solitary Sig vs Dyadic Sig": mannwhitneyu(solitary_sig["n90"], dyadic_sig["n90"]),
    "Solitary Non-Sig vs Dyadic Non-Sig": mannwhitneyu(solitary_non_sig["n90"], dyadic_non_sig["n90"]),
}

results

{'Solitary Sig vs Non-Sig': MannwhitneyuResult(statistic=np.float64(141.0), pvalue=np.float64(0.04912459928160622)),
 'Dyadic Sig vs Non-Sig': MannwhitneyuResult(statistic=np.float64(137.0), pvalue=np.float64(0.07313128420037963)),
 'Solitary Sig vs Dyadic Sig': MannwhitneyuResult(statistic=np.float64(30.0), pvalue=np.float64(0.8728638922386202)),
 'Solitary Non-Sig vs Dyadic Non-Sig': MannwhitneyuResult(statistic=np.float64(241.0), pvalue=np.float64(0.3280986919845258))}

# Supplementary Figure 4

In [46]:
skeleton_df = load_object(data_output_dir / "skeleton_df.pkl")

In [47]:
moseq_df_copy = moseq_df.rename_axis(["track", "frame_index"])
name_dyadic_bool_series = moseq_df_copy.index.get_level_values("track").unique().to_frame()["track"].str.contains("&")
dyadic_names = name_dyadic_bool_series[name_dyadic_bool_series].index

dyadic_moseq_df = moseq_df_copy.loc[pd.IndexSlice[dyadic_names, :], :]

dyadic_moseq_df_reindexed = dyadic_moseq_df.copy().reset_index()
dyadic_moseq_df_reindexed["strain"] = "Bl6"
dyadic_moseq_df_reindexed["light_cycle"] = dyadic_moseq_df_reindexed["light_cycle"].replace({"Normal": "norm", "Reverse": "reverse"})
dyadic_moseq_df_reindexed["mouse_id_string"] = dyadic_moseq_df_reindexed["track"].str.extract(r"(\d+&\d+)")
dyadic_moseq_df_reindexed["track_id"] = dyadic_moseq_df_reindexed["track"].str.extract(r"_track(\d+)")
dyadic_moseq_df_reindexed = dyadic_moseq_df_reindexed.rename({"frame_idx": "frame_index"}, axis=1)
dyadic_moseq_df_reindexed = dyadic_moseq_df_reindexed.set_index(["strain", "light_cycle", "mouse_id_string", "track_id", "frame_index"])
dyadic_moseq_df_reindexed = dyadic_moseq_df_reindexed.sort_index()
dyadic_moseq_df_reindexed

track  \
strain light_cycle mouse_id_string track_id frame_index                                    
Bl6    norm        1&4             0        756           1&4 - 19-03-24.analysis_track0   
                                            757           1&4 - 19-03-24.analysis_track0   
                                            758           1&4 - 19-03-24.analysis_track0   
                                            759           1&4 - 19-03-24.analysis_track0   
                                            760           1&4 - 19-03-24.analysis_track0   
...                                                                                  ...   
       reverse     7&10            1        30272        7&10 - 20-03-24.analysis_track1   
                                            30273        7&10 - 20-03-24.analysis_track1   
                                            30274        7&10 - 20-03-24.analysis_track1   
                                            30275        7&10 - 20-03-24.analysis_track1   
                                            30276        7&10 - 20-03-24.analysis_track1   

                                                         centroid_x  \
strain light_cycle mouse_id_string track_id frame_index               
Bl6    norm        1&4             0        756          503.740443   
                                            757          503.479071   
                                            758          503.475448   
                                            759          503.445168   
                                            760          503.403356   
...                                                             ...   
       reverse     7&10            1        30272        215.918055   
                                            30273        208.603168   
                                            30274        200.272810   
                                            30275        192.997141   
                                            30276        187.324546   

                                                         centroid_y   heading  \
strain light_cycle mouse_id_string track_id frame_index                         
Bl6    norm        1&4             0        756          451.215687  0.938396   
                                            757          451.394892  0.996951   
                                            758          452.739480  1.017735   
                                            759          453.145234  1.084564   
                                            760          453.544596  1.084564   
...                                                             ...       ...   
       reverse     7&10            1        30272        212.280512  2.944130   
                                            30273        212.501689  2.954739   
                                            30274        213.195882  2.993109   
                                            30275        213.228413  2.993109   
                                            30276        214.799292  2.972206   

                                                         syllable  \
strain light_cycle mouse_id_string track_id frame_index             
Bl6    norm        1&4             0        756                17   
                                            757                17   
                                            758                17   
                                            759                17   
                                            760                17   
...                                                           ...   
       reverse     7&10            1        30272               5   
                                            30273               5   
                                            30274               5   
                                            30275               5   
                                            30276               5   

          

## Panel A (0, 5, 9)

In [48]:
n_samples = 16
rng = np.random.default_rng(1231231)

selected_pattern = (0, 5, 9)
acceptable_distance = 0

syllable_chunks = rle_series(dyadic_moseq_df_reindexed["syllable"], slice_index=-1)
syllable_df = dyadic_moseq_df_reindexed["syllable"].unstack("frame_index").dropna(axis=1, how="any").astype(int)

pattern_match_df = get_pattern_match_df(syllable_df, patterns_to_match=[selected_pattern], hamming_distance=acceptable_distance)

syll_length_summary_df = pattern_match_df["syllable_lengths"].apply(pd.Series).describe()
minimum_syllable_length = 6

pattern_chunks = match_df_to_chunks_df(pattern_match_df, keep_pattern_cols=True)

track_name_series = dyadic_moseq_df_reindexed.groupby(["strain", "light_cycle", "mouse_id_string", "track_id"]).apply(lambda x: x.iloc[0])["track"]

filtered_pattern_chunks = pattern_chunks[pattern_chunks["syllable_lengths"].apply(lambda x: not any([length < minimum_syllable_length for length in x]))]
print(f"{len(filtered_pattern_chunks)} chunks with all syllables >= {minimum_syllable_length}. {len(pattern_chunks) - len(filtered_pattern_chunks)} chunks removed.")

sampled_pattern_chunks = filtered_pattern_chunks.sample(n=n_samples, random_state=rng)
sampled_pattern_chunks["track"] = sampled_pattern_chunks["grouped_index"].map(track_name_series)

median_length = filtered_pattern_chunks["length"].median()
print("Median length of chunks:", median_length)

sampled_pattern_trajectory_df = extract_subtrack_df(tracking_df, sampled_pattern_chunks.set_index(["track", "index_start"]).index, span_before=5, span_after=median_length+5)

corresponding_moseq_df = moseq_df.rename({"centroid_x": "x", "centroid_y": "y"}, axis=1)
corresponding_moseq_df = extract_subtrack_df(corresponding_moseq_df, sampled_pattern_chunks.set_index(["track", "index_start"]).index, span_before=5, span_after=median_length+5)

# set all values such that they reference frame_index 0, at all reference_frame_index
original_index = corresponding_moseq_df.index
corresponding_moseq_df = corresponding_moseq_df.xs(0, level="frame_index", axis=0).loc[corresponding_moseq_df.index]
corresponding_moseq_df.index = original_index

normalized_trajectory_df = normalize_track_df(sampled_pattern_trajectory_df, corresponding_moseq_df).sort_index()

100%|██████████| 1/1 [00:00<?, ?it/s]


38 chunks with all syllables >= 6. 110 chunks removed.
Median length of chunks: 41.0


In [49]:
save_object(filtered_pattern_chunks, data_output_dir / "filtered_pattern_chunks_059.pkl", overwrite=True)
save_object(normalized_trajectory_df, data_output_dir / "normalized_trajectory_df_059.pkl", overwrite=True)

## Panel B (2, 22, 2)

In [50]:
n_samples = 16
rng = np.random.default_rng(1231231)

selected_pattern = (2, 22, 2)
acceptable_distance = 0

syllable_chunks = rle_series(dyadic_moseq_df_reindexed["syllable"], slice_index=-1)
syllable_df = dyadic_moseq_df_reindexed["syllable"].unstack("frame_index").dropna(axis=1, how="any").astype(int)

pattern_match_df = get_pattern_match_df(syllable_df, patterns_to_match=[selected_pattern], hamming_distance=acceptable_distance)

syll_length_summary_df = pattern_match_df["syllable_lengths"].apply(pd.Series).describe()
minimum_syllable_length = 6

pattern_chunks = match_df_to_chunks_df(pattern_match_df, keep_pattern_cols=True)

track_name_series = dyadic_moseq_df_reindexed.groupby(["strain", "light_cycle", "mouse_id_string", "track_id"]).apply(lambda x: x.iloc[0])["track"]

filtered_pattern_chunks = pattern_chunks[pattern_chunks["syllable_lengths"].apply(lambda x: not any([length < minimum_syllable_length for length in x]))]
print(f"{len(filtered_pattern_chunks)} chunks with all syllables >= {minimum_syllable_length}. {len(pattern_chunks) - len(filtered_pattern_chunks)} chunks removed.")

sampled_pattern_chunks = filtered_pattern_chunks.sample(n=n_samples, random_state=rng)
sampled_pattern_chunks["track"] = sampled_pattern_chunks["grouped_index"].map(track_name_series)

median_length = filtered_pattern_chunks["length"].median()
print("Median length of chunks:", median_length)

sampled_pattern_trajectory_df = extract_subtrack_df(tracking_df, sampled_pattern_chunks.set_index(["track", "index_start"]).index, span_before=5, span_after=median_length+5)

corresponding_moseq_df = moseq_df.rename({"centroid_x": "x", "centroid_y": "y"}, axis=1)
corresponding_moseq_df = extract_subtrack_df(corresponding_moseq_df, sampled_pattern_chunks.set_index(["track", "index_start"]).index, span_before=5, span_after=median_length+5)

# set all values such that they reference frame_index 0, at all reference_frame_index
original_index = corresponding_moseq_df.index
corresponding_moseq_df = corresponding_moseq_df.xs(0, level="frame_index", axis=0).loc[corresponding_moseq_df.index]
corresponding_moseq_df.index = original_index

normalized_trajectory_df = normalize_track_df(sampled_pattern_trajectory_df, corresponding_moseq_df).sort_index()

100%|██████████| 1/1 [00:00<?, ?it/s]


28 chunks with all syllables >= 6. 43 chunks removed.
Median length of chunks: 39.0


In [51]:
save_object(filtered_pattern_chunks, data_output_dir / "filtered_pattern_chunks_2222.pkl", overwrite=True)
save_object(normalized_trajectory_df, data_output_dir / "normalized_trajectory_df_2222.pkl", overwrite=True)

## Panel C (21, 1, 13)

In [52]:
n_samples = 16
rng = np.random.default_rng(1231231)

selected_pattern = (21, 1, 13)
acceptable_distance = 0

syllable_chunks = rle_series(dyadic_moseq_df_reindexed["syllable"], slice_index=-1)
syllable_df = dyadic_moseq_df_reindexed["syllable"].unstack("frame_index").dropna(axis=1, how="any").astype(int)

pattern_match_df = get_pattern_match_df(syllable_df, patterns_to_match=[selected_pattern], hamming_distance=acceptable_distance)

syll_length_summary_df = pattern_match_df["syllable_lengths"].apply(pd.Series).describe()
minimum_syllable_length = 6

pattern_chunks = match_df_to_chunks_df(pattern_match_df, keep_pattern_cols=True)

track_name_series = dyadic_moseq_df_reindexed.groupby(["strain", "light_cycle", "mouse_id_string", "track_id"]).apply(lambda x: x.iloc[0])["track"]

filtered_pattern_chunks = pattern_chunks[pattern_chunks["syllable_lengths"].apply(lambda x: not any([length < minimum_syllable_length for length in x]))]
print(f"{len(filtered_pattern_chunks)} chunks with all syllables >= {minimum_syllable_length}. {len(pattern_chunks) - len(filtered_pattern_chunks)} chunks removed.")

sampled_pattern_chunks = filtered_pattern_chunks.sample(n=n_samples, random_state=rng)
sampled_pattern_chunks["track"] = sampled_pattern_chunks["grouped_index"].map(track_name_series)

median_length = filtered_pattern_chunks["length"].median()
print("Median length of chunks:", median_length)

sampled_pattern_trajectory_df = extract_subtrack_df(tracking_df, sampled_pattern_chunks.set_index(["track", "index_start"]).index, span_before=5, span_after=median_length+5)

corresponding_moseq_df = moseq_df.rename({"centroid_x": "x", "centroid_y": "y"}, axis=1)
corresponding_moseq_df = extract_subtrack_df(corresponding_moseq_df, sampled_pattern_chunks.set_index(["track", "index_start"]).index, span_before=5, span_after=median_length+5)

# set all values such that they reference frame_index 0, at all reference_frame_index
original_index = corresponding_moseq_df.index
corresponding_moseq_df = corresponding_moseq_df.xs(0, level="frame_index", axis=0).loc[corresponding_moseq_df.index]
corresponding_moseq_df.index = original_index

normalized_trajectory_df = normalize_track_df(sampled_pattern_trajectory_df, corresponding_moseq_df).sort_index()

100%|██████████| 1/1 [00:00<?, ?it/s]


37 chunks with all syllables >= 6. 32 chunks removed.
Median length of chunks: 49.0


In [53]:
save_object(filtered_pattern_chunks, data_output_dir / "filtered_pattern_chunks_21113.pkl", overwrite=True)
save_object(normalized_trajectory_df, data_output_dir / "normalized_trajectory_df_21113.pkl", overwrite=True)

## Panel D (2, 8, 29)

In [54]:
n_samples = 16
rng = np.random.default_rng(1231231)

selected_pattern = (2, 8, 29)
acceptable_distance = 0

syllable_chunks = rle_series(dyadic_moseq_df_reindexed["syllable"], slice_index=-1)
syllable_df = dyadic_moseq_df_reindexed["syllable"].unstack("frame_index").dropna(axis=1, how="any").astype(int)

pattern_match_df = get_pattern_match_df(syllable_df, patterns_to_match=[selected_pattern], hamming_distance=acceptable_distance)

syll_length_summary_df = pattern_match_df["syllable_lengths"].apply(pd.Series).describe()
minimum_syllable_length = 6

pattern_chunks = match_df_to_chunks_df(pattern_match_df, keep_pattern_cols=True)

track_name_series = dyadic_moseq_df_reindexed.groupby(["strain", "light_cycle", "mouse_id_string", "track_id"]).apply(lambda x: x.iloc[0])["track"]

filtered_pattern_chunks = pattern_chunks[pattern_chunks["syllable_lengths"].apply(lambda x: not any([length < minimum_syllable_length for length in x]))]
print(f"{len(filtered_pattern_chunks)} chunks with all syllables >= {minimum_syllable_length}. {len(pattern_chunks) - len(filtered_pattern_chunks)} chunks removed.")

sampled_pattern_chunks = filtered_pattern_chunks.sample(n=n_samples, random_state=rng)
sampled_pattern_chunks["track"] = sampled_pattern_chunks["grouped_index"].map(track_name_series)

median_length = filtered_pattern_chunks["length"].median()
print("Median length of chunks:", median_length)

sampled_pattern_trajectory_df = extract_subtrack_df(tracking_df, sampled_pattern_chunks.set_index(["track", "index_start"]).index, span_before=5, span_after=median_length+5)

corresponding_moseq_df = moseq_df.rename({"centroid_x": "x", "centroid_y": "y"}, axis=1)
corresponding_moseq_df = extract_subtrack_df(corresponding_moseq_df, sampled_pattern_chunks.set_index(["track", "index_start"]).index, span_before=5, span_after=median_length+5)

# set all values such that they reference frame_index 0, at all reference_frame_index
original_index = corresponding_moseq_df.index
corresponding_moseq_df = corresponding_moseq_df.xs(0, level="frame_index", axis=0).loc[corresponding_moseq_df.index]
corresponding_moseq_df.index = original_index

normalized_trajectory_df = normalize_track_df(sampled_pattern_trajectory_df, corresponding_moseq_df).sort_index()

100%|██████████| 1/1 [00:00<?, ?it/s]


45 chunks with all syllables >= 6. 101 chunks removed.
Median length of chunks: 52.0


In [55]:
save_object(filtered_pattern_chunks, data_output_dir / "filtered_pattern_chunks_2829.pkl", overwrite=True)
save_object(normalized_trajectory_df, data_output_dir / "normalized_trajectory_df_2829.pkl", overwrite=True)

# Supplementary Figure 5

In [56]:
group_dict = load_group_dict_from_csv(moseq_project_dir / "index.csv")
name_group_df = pd.Series(group_dict).str.split(", ", expand=True)
name_group_df.columns = ["treatment", "context", "light_cycle", "shave_status"]

moseq_df

centroid_x  centroid_y   heading  \
track                           frame_index                                     
1 - 21-02-24.analysis           756           78.633054  409.166669  2.442800   
                                757           78.633054  412.106139  2.204218   
                                758           78.052170  414.912978  2.185359   
                                759           77.818435  417.413945  2.136504   
                                760           77.706140  419.782589  2.105274   
...                                                 ...         ...       ...   
7&10 - 20-03-24.analysis_track1 30272        215.918055  212.280512  2.944130   
                                30273        208.603168  212.501689  2.954739   
                                30274        200.272810  213.195882  2.993109   
                                30275        192.997141  213.228413  2.993109   
                                30276        187.324546  214.799292  2.972206   

                                             syllable  \
track                           frame_index             
1 - 21-02-24.analysis           756                 1   
                                757                 1   
                                758                13   
                                759                13   
                                760                13   
...                                               ...   
7&10 - 20-03-24.analysis_track1 30272               5   
                                30273               5   
                                30274               5   
                                30275               5   
                                30276               5   

                                                                          group  \
track                           frame_index                                       
1 - 21-02-24.analysis           756                   Vehicle, Solitary, Normal   
                                757                   Vehicle, Solitary, Normal   
                                758                   Vehicle, Solitary, Normal   
                                759                   Vehicle, Solitary, Normal   
                                760                   Vehicle, Solitary, Normal   
...                                                                         ...   
7&10 - 20-03-24.analysis_track1 30272        Vehicle, Dyadic, Reverse, Unshaved   
                                30273        Vehicle, Dyadic, Reverse, Unshaved   
                                30274        Vehicle, Dyadic, Reverse, Unshaved   
                                30275        Vehicle, Dyadic, Reverse, Unshaved   
                                30276        Vehicle, Dyadic, Reverse, Unshaved   

                                             velocity_px_s  angular_velocity  \
track                           frame_index                                    
1 - 21-02-24.analysis           756              88.265142         -2.977538   
                                757             101.147777         -2.632186   
                                758              85.989485         -2.216982   
                                759              75.355969         -1.784897   
                                760              71.139130         -1.388754   
...                                                    ...               ...   
7&10 - 20-03-24.analysis_track1 30272           243.246585          0.112511   
                                30273           219.546911          0.145722   
                                30274           250.776992          0.159203   
                                30275           218.272241          0.158475   
                                30276           179.456493          0.152606   

                                             onset  syllable_frame  \
track                           frame_index               

## Panel A (9, 0, 5) Family

In [57]:
n_samples = 16
rng = np.random.default_rng(1231231)

selected_pattern = (9, 0, 5)
acceptable_distance = 1

syllable_chunks = rle_series(dyadic_moseq_df_reindexed["syllable"], slice_index=-1)
syllable_df = dyadic_moseq_df_reindexed["syllable"].unstack("frame_index").dropna(axis=1, how="any").astype(int)

pattern_match_df = get_pattern_match_df(syllable_df, patterns_to_match=[selected_pattern], hamming_distance=acceptable_distance)

syll_length_summary_df = pattern_match_df["syllable_lengths"].apply(pd.Series).describe()
minimum_syllable_length = 6

pattern_chunks = match_df_to_chunks_df(pattern_match_df, keep_pattern_cols=True)

track_name_series = dyadic_moseq_df_reindexed.groupby(["strain", "light_cycle", "mouse_id_string", "track_id"]).apply(lambda x: x.iloc[0])["track"]

filtered_pattern_chunks = pattern_chunks[pattern_chunks["syllable_lengths"].apply(lambda x: not any([length < minimum_syllable_length for length in x]))]
print(f"{len(filtered_pattern_chunks)} chunks with all syllables >= {minimum_syllable_length}. {len(pattern_chunks) - len(filtered_pattern_chunks)} chunks removed.")

sampled_pattern_chunks = filtered_pattern_chunks.sample(n=n_samples, random_state=rng)
sampled_pattern_chunks["track"] = sampled_pattern_chunks["grouped_index"].map(track_name_series)

median_length = filtered_pattern_chunks["length"].median()
print("Median length of chunks:", median_length)

sampled_pattern_trajectory_df = extract_subtrack_df(tracking_df, sampled_pattern_chunks.set_index(["track", "index_start"]).index, span_before=5, span_after=median_length+5)

corresponding_moseq_df = moseq_df.rename({"centroid_x": "x", "centroid_y": "y"}, axis=1)
corresponding_moseq_df = extract_subtrack_df(corresponding_moseq_df, sampled_pattern_chunks.set_index(["track", "index_start"]).index, span_before=5, span_after=median_length+5)

# set all values such that they reference frame_index 0, at all reference_frame_index
original_index = corresponding_moseq_df.index
corresponding_moseq_df = corresponding_moseq_df.xs(0, level="frame_index", axis=0).loc[corresponding_moseq_df.index]
corresponding_moseq_df.index = original_index

normalized_trajectory_df = normalize_track_df(sampled_pattern_trajectory_df, corresponding_moseq_df).sort_index()

100%|██████████| 1/1 [00:00<?, ?it/s]


624 chunks with all syllables >= 6. 1417 chunks removed.
Median length of chunks: 44.0


In [58]:
save_object(filtered_pattern_chunks, data_output_dir / "filtered_pattern_chunks_905_family.pkl", overwrite=True)
save_object(normalized_trajectory_df, data_output_dir / "normalized_trajectory_df_905_family.pkl", overwrite=True)

## Panel B (11, 2, 8) Family

In [59]:
n_samples = 16
rng = np.random.default_rng(1231231)

selected_pattern = (11, 2, 8)
acceptable_distance = 1

syllable_chunks = rle_series(dyadic_moseq_df_reindexed["syllable"], slice_index=-1)
syllable_df = dyadic_moseq_df_reindexed["syllable"].unstack("frame_index").dropna(axis=1, how="any").astype(int)

pattern_match_df = get_pattern_match_df(syllable_df, patterns_to_match=[selected_pattern], hamming_distance=acceptable_distance)

syll_length_summary_df = pattern_match_df["syllable_lengths"].apply(pd.Series).describe()
minimum_syllable_length = 6

pattern_chunks = match_df_to_chunks_df(pattern_match_df, keep_pattern_cols=True)

track_name_series = dyadic_moseq_df_reindexed.groupby(["strain", "light_cycle", "mouse_id_string", "track_id"]).apply(lambda x: x.iloc[0])["track"]

filtered_pattern_chunks = pattern_chunks[pattern_chunks["syllable_lengths"].apply(lambda x: not any([length < minimum_syllable_length for length in x]))]
print(f"{len(filtered_pattern_chunks)} chunks with all syllables >= {minimum_syllable_length}. {len(pattern_chunks) - len(filtered_pattern_chunks)} chunks removed.")

sampled_pattern_chunks = filtered_pattern_chunks.sample(n=n_samples, random_state=rng)
sampled_pattern_chunks["track"] = sampled_pattern_chunks["grouped_index"].map(track_name_series)

median_length = filtered_pattern_chunks["length"].median()
print("Median length of chunks:", median_length)

sampled_pattern_trajectory_df = extract_subtrack_df(tracking_df, sampled_pattern_chunks.set_index(["track", "index_start"]).index, span_before=5, span_after=median_length+5)

corresponding_moseq_df = moseq_df.rename({"centroid_x": "x", "centroid_y": "y"}, axis=1)
corresponding_moseq_df = extract_subtrack_df(corresponding_moseq_df, sampled_pattern_chunks.set_index(["track", "index_start"]).index, span_before=5, span_after=median_length+5)

# set all values such that they reference frame_index 0, at all reference_frame_index
original_index = corresponding_moseq_df.index
corresponding_moseq_df = corresponding_moseq_df.xs(0, level="frame_index", axis=0).loc[corresponding_moseq_df.index]
corresponding_moseq_df.index = original_index

normalized_trajectory_df = normalize_track_df(sampled_pattern_trajectory_df, corresponding_moseq_df).sort_index()

100%|██████████| 1/1 [00:00<?, ?it/s]


499 chunks with all syllables >= 6. 1202 chunks removed.
Median length of chunks: 45.0


In [60]:
save_object(filtered_pattern_chunks, data_output_dir / "filtered_pattern_chunks_1128_family.pkl", overwrite=True)
save_object(normalized_trajectory_df, data_output_dir / "normalized_trajectory_df_1128_family.pkl", overwrite=True)

# Supplementary Figure 6

## Panel A

In [61]:
# statistical test 2: proportion of 905 1128 from approach, leave chi2 proportiuo ntest then go tocontact, that should be lower (even if significant)... what is a good abseline?

pattern_match_df_copy = load_object(data_output_dir / "pattern_match_df_copy.pkl")
global_match_df_copy = load_object(data_output_dir / "global_match_df_copy.pkl")
clean_polygon_dict = load_object(data_output_dir / "clean_polygon_dict.pkl")
pattern_match_df_copy

match  \
pattern strain light_cycle mouse_id_string track_id match_idx         
lci     Bl6    norm        1&4             0        0           lci   
                                                    1           lcw   
                                                    2           lci   
                                                    3           yci   
                                                    4           qci   
...                                                             ...   
jaf     Bl6    reverse     7&10            1        70          paf   
                                                    71          jaf   
                                                    72          paf   
                                                    73          ja   
                                                    74          xaf   

                                                              mismatches  \
pattern strain light_cycle mouse_id_string track_id match_idx              
lci     Bl6    norm        1&4             0        0                 []   
                                                    1                [2]   
                                                    2                 []   
                                                    3                [0]   
                                                    4                [0]   
...                                                                  ...   
jaf     Bl6    reverse     7&10            1        70               [0]   
                                                    71                []   
                                                    72               [0]   
                                                    73               [2]   
                                                    74               [0]   

                                                               match_start  \
pattern strain light_cycle mouse_id_string track_id match_idx                
lci     Bl6    norm        1&4             0        0                   13   
                                                    1                   35   
                                                    2                   38   
                                                    3                   74   
                                                    4                   99   
...                                                                    ...   
jaf     Bl6    reverse     7&10            1        70                1730   
                                                    71                1738   
                                                    72                1752   
                                                    73                1762   
                                                    74                1775   

                                                               match_end  \
pattern strain light_cycle mouse_id_string track_id match_idx              
lci     Bl6    norm        1&4             0        0                 16   
                                                    1                 38   
                                                    2                 41   
                                                    3                 77   
                                                    4                102   
...                                                                  ...   
jaf     Bl6    reverse     7&10            1        70              1733   
                                                    71              1741   
                                                    72              1755   
                                                    73              1765   
                                                    74              1778   

                                                              syllable_match  \
pattern strain light_cycle mouse_id_string track_id match_idx    

In [62]:
polygon_series = pd.Series(clean_polygon_dict)

pattern_match_polygon_check = pattern_match_df_copy[["start_distance", "distance_difference"]].apply(
    lambda x: polygon_series.apply(
        lambda y: y.contains(Point(x["start_distance"], x["distance_difference"]))
        ), axis=1)

containing_polygon_series = pattern_match_polygon_check.rename_axis("containing_polygon", axis=1).stack().rename("contains_point")
containing_polygon_series = containing_polygon_series[containing_polygon_series].reset_index("containing_polygon").drop("contains_point", axis=1).squeeze()
pattern_match_df_copy = pattern_match_df_copy.join(containing_polygon_series)
pattern_match_df_copy["only_relevant_sylls"] = pattern_match_df_copy["syllable_match"].apply(lambda x: all(syllable in relevant_syllables for syllable in x))
pattern_match_df_copy

match  \
pattern strain light_cycle mouse_id_string track_id match_idx         
lci     Bl6    norm        1&4             0        0           lci   
                                                    1           lcw   
                                                    2           lci   
                                                    3           yci   
                                                    4           qci   
...                                                             ...   
jaf     Bl6    reverse     7&10            1        70          paf   
                                                    71          jaf   
                                                    72          paf   
                                                    73          ja   
                                                    74          xaf   

                                                              mismatches  \
pattern strain light_cycle mouse_id_string track_id match_idx              
lci     Bl6    norm        1&4             0        0                 []   
                                                    1                [2]   
                                                    2                 []   
                                                    3                [0]   
                                                    4                [0]   
...                                                                  ...   
jaf     Bl6    reverse     7&10            1        70               [0]   
                                                    71                []   
                                                    72               [0]   
                                                    73               [2]   
                                                    74               [0]   

                                                               match_start  \
pattern strain light_cycle mouse_id_string track_id match_idx                
lci     Bl6    norm        1&4             0        0                   13   
                                                    1                   35   
                                                    2                   38   
                                                    3                   74   
                                                    4                   99   
...                                                                    ...   
jaf     Bl6    reverse     7&10            1        70                1730   
                                                    71                1738   
                                                    72                1752   
                                                    73                1762   
                                                    74                1775   

                                                               match_end  \
pattern strain light_cycle mouse_id_string track_id match_idx              
lci     Bl6    norm        1&4             0        0                 16   
                                                    1                 38   
                                                    2                 41   
                                                    3                 77   
                                                    4                102   
...                                                                  ...   
jaf     Bl6    reverse     7&10            1        70              1733   
                                                    71              1741   
                                                    72              1755   
                                                    73              1765   
                                                    74              1778   

                                                              syllable_match  \
pattern strain light_cycle mouse_id_string track_id match_idx    

In [63]:
polygon_series = pd.Series(clean_polygon_dict)

pattern_match_polygon_check = global_match_df_copy[["start_distance", "distance_difference"]].apply(
    lambda x: polygon_series.apply(
        lambda y: y.contains(Point(x["start_distance"], x["distance_difference"]))
        ), axis=1)

containing_polygon_series = pattern_match_polygon_check.rename_axis("containing_polygon", axis=1).stack().rename("contains_point")
containing_polygon_series = containing_polygon_series[containing_polygon_series].reset_index("containing_polygon").drop("contains_point", axis=1).squeeze()
global_match_df_copy = global_match_df_copy.join(containing_polygon_series)
global_match_df_copy["only_relevant_sylls"] = global_match_df_copy["syllable_match"].apply(lambda x: all(syllable in relevant_syllables for syllable in x))
global_match_df_copy

match  \
pattern strain light_cycle mouse_id_string track_id match_idx         
aa     Bl6    norm        1&4             0        0           aa   
                                                    1           aa   
                                                    2           aa   
                                                    3           aa   
                                           1        0           aa   
...                                                             ...   
j     Bl6    norm        1&4             1        0           j   
l}     Bl6    reverse     25&28           1        0           l}   
     Bl6    reverse     25&28           1        0              
     Bl6    norm        7&10            0        0              
d     Bl6    reverse     13&16           1        0           d   

                                                              mismatches  \
pattern strain light_cycle mouse_id_string track_id match_idx              
aa     Bl6    norm        1&4             0        0                 []   
                                                    1                 []   
                                                    2                 []   
                                                    3                 []   
                                           1        0                 []   
...                                                                  ...   
j     Bl6    norm        1&4             1        0                 []   
l}     Bl6    reverse     25&28           1        0                 []   
     Bl6    reverse     25&28           1        0                 []   
     Bl6    norm        7&10            0        0                 []   
d     Bl6    reverse     13&16           1        0                 []   

                                                               match_start  \
pattern strain light_cycle mouse_id_string track_id match_idx                
aa     Bl6    norm        1&4             0        0                  993   
                                                    1                 1120   
                                                    2                 1437   
                                                    3                 1801   
                                           1        0                  115   
...                                                                    ...   
j     Bl6    norm        1&4             1        0                 1663   
l}     Bl6    reverse     25&28           1        0                  367   
     Bl6    reverse     25&28           1        0                 1680   
     Bl6    norm        7&10            0        0                 1670   
d     Bl6    reverse     13&16           1        0                 1302   

                                                               match_end  \
pattern strain light_cycle mouse_id_string track_id match_idx              
aa     Bl6    norm        1&4             0        0                996   
                                                    1               1123   
                                                    2               1440   
                                                    3               1804   
                                           1        0                118   
...                                                                  ...   
j     Bl6    norm        1&4             1        0               1666   
l}     Bl6    reverse     25&28           1        0                370   
     Bl6    reverse     25&28           1        0               1683   
     Bl6    norm        7&10            0        0               1673   
d     Bl6    reverse     13&16           1        0               1305   

                                                              syllable_match  \
pattern strain light_cycle mouse_id_string track_id match_idx    

In [64]:
pattern_proportion_df = pattern_match_df_copy.groupby("syllable_pattern")["containing_polygon"].value_counts(normalize=False, dropna=False).rename("occurrence_inside").to_frame()
pattern_proportion_df.index = pd.MultiIndex.from_frame(pattern_proportion_df.index.to_frame().astype(str))
pattern_proportion_df["occurrence_outside"] = pattern_proportion_df["occurrence_inside"].groupby("syllable_pattern").apply(lambda x: x.sum() - x).droplevel(0)

global_pattern_proportion_df = global_match_df_copy["containing_polygon"].value_counts(normalize=False, dropna=False).rename("occurrence_inside").to_frame()
global_pattern_proportion_df.index = pd.MultiIndex.from_frame(global_pattern_proportion_df.index.to_frame().astype(str))
global_pattern_proportion_df["occurrence_outside"] = global_pattern_proportion_df["occurrence_inside"].sum() - global_pattern_proportion_df["occurrence_inside"]
global_pattern_proportion_df

,occurrence_inside,occurrence_outside
containing_polygon,,
stationary_no_contact,12894,26673
nan,9961,29606
contact,6346,33221
targeted_approach,3503,36064
targeted_leave,3484,36083
leave_control,1739,37828
approach_control,1640,37927


In [65]:
results_ratio_dict = {}
results_p_dict = {}
for test_polygon in polygon_series.keys():
    contingency_pattern905 = pattern_proportion_df.loc[pd.IndexSlice["(9, 0, 5)", test_polygon], :]
    contingency_pattern1128 = pattern_proportion_df.loc[pd.IndexSlice["(11, 2, 8)", test_polygon], :]
    contingency_global = global_pattern_proportion_df.loc[test_polygon].squeeze()
    contingency_global.name = ("all", test_polygon)

    contingency_table = pd.concat([contingency_pattern905, contingency_global], axis=1).T
    chi2, p, dof, exp_prop = chi2_contingency(contingency_table.values)
    results_p_dict[(test_polygon, "(9, 0, 5)")] = p

    contingency_table = pd.concat([contingency_pattern1128, contingency_global], axis=1).T
    chi2, p, dof, exp_prop = chi2_contingency(contingency_table.values)
    results_p_dict[(test_polygon, "(11, 2, 8)")] = p

    ratio_table = pd.concat([contingency_pattern905, contingency_pattern1128, contingency_global], axis=1).T
    results_ratio_dict[test_polygon] = (ratio_table["occurrence_inside"] / ratio_table["occurrence_outside"]).droplevel(1)

In [66]:
results_ratio_series = pd.concat(results_ratio_dict, names=["containing_polygon", "syllable_pattern"])
pvalue_series = pd.Series(results_p_dict)
results_ratio_series

containing_polygon     syllable_pattern
contact                (9, 0, 5)           0.133889
                       (11, 2, 8)          0.106701
                       all                 0.191024
targeted_leave         (9, 0, 5)           0.159001
                       (11, 2, 8)          0.164271
                       all                 0.096555
leave_control          (9, 0, 5)           0.064684
                       (11, 2, 8)          0.053903
                       all                 0.045971
targeted_approach      (9, 0, 5)           0.142777
                       (11, 2, 8)          0.120553
                       all                 0.097133
approach_control       (9, 0, 5)           0.075342
                       (11, 2, 8)          0.086901
                       all                 0.043241
stationary_no_contact  (9, 0, 5)           0.251380
                       (11, 2, 8)          0.292553
                       all                 0.483410
dtype: float64

In [67]:
save_object(results_ratio_series, data_output_dir / "syllable_family_polygon_ratio_series.pkl", overwrite=True)

In [68]:
print(len(pvalue_series))
pvalue_series.loc[["contact", "stationary_no_contact", "targeted_approach", "approach_control", "targeted_leave", "leave_control"]] < (0.05 / len(pvalue_series))

12


contact                (9, 0, 5)      True
                       (11, 2, 8)     True
stationary_no_contact  (9, 0, 5)      True
                       (11, 2, 8)     True
targeted_approach      (9, 0, 5)      True
                       (11, 2, 8)    False
approach_control       (9, 0, 5)      True
                       (11, 2, 8)     True
targeted_leave         (9, 0, 5)      True
                       (11, 2, 8)     True
leave_control          (9, 0, 5)      True
                       (11, 2, 8)    False
dtype: bool

In [69]:
test_polygon1 = "targeted_approach"
test_polygon2 = "approach_control"
contingency1_pattern = pattern_proportion_df.loc[pd.IndexSlice["(9, 0, 5)", test_polygon1], :].astype(float)
contingency2_pattern = pattern_proportion_df.loc[pd.IndexSlice["(9, 0, 5)", test_polygon2], :].astype(float)
contingency2_pattern_sum = contingency2_pattern.sum()
contingency2_pattern["occurrence_inside"] *= (polygon_series[test_polygon1].area / polygon_series[test_polygon2].area)
contingency2_pattern["occurrence_outside"] = contingency2_pattern_sum - contingency2_pattern["occurrence_inside"]

contingency_table = pd.concat([contingency1_pattern, contingency2_pattern], axis=1).T
chi2, p, dof, exp_prop = chi2_contingency(contingency_table.values)
print(chi2, p, dof, exp_prop)
print(contingency_table - exp_prop)

17.849623833458146 2.3906652295838882e-05 1 [[ 213.21875 1827.78125]
 [ 213.21875 1827.78125]]
                             occurrence_inside  occurrence_outside
(9, 0, 5) targeted_approach           41.78125           -41.78125
          approach_control           -41.78125            41.78125


## Panel B

In [70]:
# already created during Step 3

## Panel C

In [71]:
# already created during Step 3